In [1]:
import time
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from utilities_s7 import *
from utilities_s8 import *

## 7 Incorporating Conditional Information

In [47]:
# Set k
k = .5

# Set model paramters
tol = 1e-12
max_iter = 1000

solver_s7 = StaDivConstraint(tol,max_iter)

### 7.1 Approach I

#### Basic problem:

\begin{equation}
\min_{M\geq0}{\mathbb E}\left[Mg(X)\right]
\end{equation}
*subject to constraints:*
\begin{align*}
&{\mathbb E}\left[M \log M\right] \leq \kappa, \\
&{\mathbb E}\left[B^j M Y\right] = 0, \\
&{\mathbb E}\left[M\right] = 1.
\end{align*}

#### Dual problem:

For computational purposes, we solve the dual problem after minimizing over $M$.  

\begin{equation*}
\sup_{\xi>0}\max_{\hat{\lambda}_j}    - \xi \log {\mathbb E} \left[ \exp\left( - {\frac 1 {\xi}} \left[ g(X_1) + \sum_{j}\hat{\lambda}_j \cdot YB^j \right] \right)\right]  -  \xi \kappa  
\end{equation*}

$\hat{\lambda}$ and $\xi$ are multipliers on the moment condition and relative entropy constraints. 

In [49]:
solver_s7.solve(k,approach=1)

{'result': -0.003154222876988816,
 'success': True,
 'message': b'CONVERGENCE: REL_REDUCTION_OF_F_<=_FACTR*EPSMCH',
 'nit': 215,
 'ξ': 0.01139072363622233,
 'λ': array([-41.92752239,  43.9329023 ,  -1.89573135,   0.06765741,
        -42.53722984,  44.54534309,  -4.14394679,  -9.59885735,
        -40.80976131,  42.81460188,  -4.95117194,  -4.46182306])}

### 7.2 Approach II

#### Basic problem:

\begin{equation}
\min_{M_1\geq0}{\mathbb E}\left[M_1g(X_1)\right]
\end{equation}
*subject to constraints:*
\begin{align*}
&{\mathbb E}\left[M_1 \log M_1\right] \leq \kappa, \\
&{\mathbb E}\left[B^j M_1 Y_1\right] = 0, \\
&{\mathbb E}\left[B^j M_1-B^j\right] = 0.
\end{align*}

#### Dual problem:

For computational purposes, we solve the dual problem after minimizing over $M_1$.  

\begin{equation*}
\sup_{\xi>0}\max_{\hat{\lambda}_j}    - \xi \sum_{j}\mathbb E[B^j]\log {\mathbb E} \left[ \exp\left( - {\frac 1 {\xi}} \left[ g(X_1) + \hat{\lambda}_j \cdot Y_1 \right] \right)\right]  -  \xi \kappa  
\end{equation*}

$\hat{\lambda}$ and $\xi$ are multipliers on the moment condition and relative entropy constraints. 

In [50]:
solver_s7.solve(k,approach=2)

{'result': -0.0024944273503736147,
 'success': True,
 'message': b'CONVERGENCE: REL_REDUCTION_OF_F_<=_FACTR*EPSMCH',
 'nit': 205,
 'ξ': 0.010298853066474024,
 'λ': array([-46.54706704,  48.54715479,  -2.3126995 ,  -0.26138998,
        -47.33513665,  49.33582501,  -5.13699403, -11.05599556,
        -45.62418906,  47.62300862,  -5.46868445,  -3.53841202])}

### 7.3 Approach III

#### Basic problem:

\begin{equation}
\min_{M_1\geq0}{\mathbb E}\left[M_1g(X_1)\mid\mathfrak{F}_0\right]
\end{equation}
*subject to constraints:*
\begin{align*}
&{\mathbb E}\left[M_1 \log M_1\mid\mathfrak{F}_0\right] \leq \kappa, \\
&{\mathbb E}\left[M_1  Y_1\mid\mathfrak{F}_0\right] = 0, \\
&{\mathbb E}\left[M_1\mid\mathfrak{F}_0\right] = 1.
\end{align*}

#### Dual problem:
For computational purposes, we solve the dual problem after minimizing over $M_1$.  

\begin{equation*}
\sup_{\xi>0}\max_{\hat{\lambda}}    - \xi \log {\mathbb E} \left[ \exp\left( - {\frac 1 {\xi}} \left[ g(X_1) + \hat{\lambda} \cdot Y_1 \right] \right)\mid\mathfrak{F}_0\right]  -  \xi \kappa  
\end{equation*}

$\hat{\lambda}$ and $\xi$ are multipliers on the moment condition and relative entropy constraints. 

In [51]:
solver_s7.solve(k,approach=3,state=3)

{'result': 0.001640716366361136,
 'success': True,
 'message': b'CONVERGENCE: REL_REDUCTION_OF_F_<=_FACTR*EPSMCH',
 'nit': 93,
 'ξ': 0.010568151037177061,
 'λ': array([-44.41474256,  46.41404401,  -5.66063623,  -3.37964178])}

## 8 Intertemporal Divergence Constraints

### Proposition 8.6
Problem 8.4 can be solved by finding the solution to:

\begin{equation}
\epsilon = \min_\hat{\lambda}\mathbb E \left(\exp \left[-\frac{1}{\xi}g(X_1)+\hat{\lambda}\cdot f(X_1)\right]\left( \frac{e_1}{e_0}\right) \mid \mathfrak{F}_0\right)
\end{equation}

*where*
\begin{align*}
\mu &= -\xi \log \epsilon,\\
v_0 &= -\xi \log e_0.
\end{align*}

Denote $e^*$, $\hat{\lambda}^*$ as the solution to the above optimization problem. The implied solution for the probablity distortion is:

\begin{equation}
M_1^* = \frac{\exp \left[-\frac{1}{\xi}g(X_1)+\hat{\lambda}^*(Z_0)\cdot f(X_1)\right]e_1^*}{\epsilon^*e_0^*}
\end{equation}

In [38]:
# Set ξ
ξ = 1.

# Set model paramters
tol = 1e-9
max_iter = 1000

solver_s8 = InterDivConstraint(tol,max_iter)

time_start = time.time() 
result = solver_s8.iterate(ξ)

# Print iteration information
print("--- Iteration Ends ---")
print("ξ = %s" % ξ)
print("Time spent: %s seconds ---" % (round(time.time()-time_start,4)))
print("Numer of iterations: %s ---" % result['count'])

# Print converged parameter results
print("\n")
print("--- Converged parameter vlues ---")
print("ϵ: %s" % result['ϵ'])
print("e: %s" % result['e'])
print("λ: %s" % result['λ'])
print("μ: %s" % result['μ'])

# Print E[M|state k]
print("\n")
print("--- Check 1 ---")
print("E[M|state 1] = %s " % result['E_M_cond'][0])
print("E[M|state 2] = %s " % result['E_M_cond'][0])
print("E[M|state 3] = %s " % result['E_M_cond'][0])

# Print two ways of calculating moment bound
print("\n")
print("--- Check 2 ---")
print("E[Mg(X)] = %s (directly using M)" % result['moment_bound'])
print("E[Mg(X)] = %s (indirectly using μ and RE)" % result['moment_bound_check'])
print("Difference: %s" % (result['moment_bound']-result['moment_bound_check']))

# Print transition probability matrix
print("\n")
print("--- Transition Probability Matrix ---")
print(result['P'])

# Print stationary distribution
print("\n")
print("--- Stationary Distribution ---")
print(result['π'])

# Print conditional relative entropy
print("\n")
print("--- Conditional Relative Entropy ---")
print("E[MlogM|state 1] = %s " % result['RE_cond'][0])
print("E[MlogM|state 2] = %s " % result['RE_cond'][1])
print("E[MlogM|state 3] = %s " % result['RE_cond'][2])

# Print unconditional relative entropy
print("\n")
print("--- Unconditional Relative Entropy ---")
print("E[MlogM] = %s " % result['RE'])

# Print conditional moment bound
print("\n")
print("--- Conditional Moment Bound ---")
print("E[Mg(X)|state 1] = %s " % result['moment_bound_cond'][0])
print("E[Mg(X)|state 2] = %s " % result['moment_bound_cond'][1])
print("E[Mg(X)|state 3] = %s " % result['moment_bound_cond'][2])

# Print unconditional moment bound
print("\n")
print("--- Unconditional Moment Bound ---")
print("E[Mg(X)] = %s " % result['moment_bound'])

--- Iteration Ends ---
ξ = 1.0
Time spent: 0.272 seconds ---
Numer of iterations: 208 ---


--- Converged parameter vlues ---
ϵ: 0.9662646505331379
e: [1.         0.44191813 0.19224315]
λ: [ 1.20464269  0.79535731 -0.31081682 -0.76069675  1.42671132  0.57328868
  0.66140712 -5.97031608  2.68482733 -0.68482733 -3.48386646 -7.80331602]
μ: 0.034317516936058806


--- Check 1 ---
E[M|state 1] = 0.9999999999543142 
E[M|state 2] = 0.9999999999543142 
E[M|state 3] = 0.9999999999543142 


--- Check 2 ---
E[Mg(X)] = 0.00575620608160032 (directly using M)
E[Mg(X)] = 0.005755997437009986 (indirectly using μ and RE)
Difference: 2.0864459033403948e-07


--- Transition Probability Matrix ---
[[0.9795479  0.0204521  0.        ]
 [0.0840223  0.88101517 0.03496253]
 [0.         0.17514752 0.82485248]]


--- Stationary Distribution ---
[0.7739925  0.18839962 0.03760788]


--- Conditional Relative Entropy ---
E[MlogM|state 1] = 0.012404221376960935 
E[MlogM|state 2] = 0.06706674330664886 
E[MlogM|state 3]

In [39]:
# Solve the minimization problems over a grid of ξ
tol = 1e-9
max_iter = 1000

solver_s8 = InterDivConstraint(tol,max_iter)

# Grid for ξ
ξ_grid = np.arange(.01,1.,.01)
μs = np.zeros_like(ξ_grid)
REs = np.zeros_like(ξ_grid)
bounds = np.zeros_like(ξ_grid)
ϵs = np.zeros_like(ξ_grid)

time_start = time.time() 
for i in range(len(ξ_grid)):
    ξ = ξ_grid[i]
    result = solver_s8.iterate(ξ)
    μs[i] = result['μ']
    REs[i] = result['RE']
    bounds[i] = result['moment_bound']
    ϵs[i] = result['ϵ']

print("Time spent: %s seconds ---" % (round(time.time()-time_start,4)))

Time spent: 25.5999 seconds ---


In [40]:
# Plots for μ and RE
fig = make_subplots(rows=1, cols=2)
fig.add_trace(
    go.Scatter(x=ξ_grid, y=bounds, name='E[Mg(X)]'),
    row=1, col=1
)
fig.add_trace(
    go.Scatter(x=ξ_grid, y=REs, name='RE'),
    row=1, col=2
)
fig.update_layout(height=400, width=1000,title_text='Plots over ξ')
fig.update_xaxes(rangemode="tozero",title_text='ξ')
fig.update_yaxes(rangemode="tozero")
fig.show()